In [ ]:
import os

In [ ]:
os.environ

In [ ]:
migs= 'MIG-c3ce33ce-ced8-5961-bb87-2b40eb100277,MIG-280489c4-1d98-5b07-b4f6-2fc85fc874fa,MIG-c432df19-0894-5232-ac1c-9a3440fc267e,MIG-e8f61a95-352a-56cc-b95d-0c35fc14e8bf'

In [ ]:
env = dict(os.environ)

In [ ]:
env['CUDA_VISIBLE_DEVICES']=migs

In [ ]:
os.environ = env

In [ ]:
os.environ

In [1]:
import os
os.environ['CUBLAS_WORKSPACE_CONFIG'] = ':5120:1'

In [2]:

os.environ['PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION'] = 'python'


In [3]:
#import os
import ray

# Set CUDA_VISIBLE_DEVICES to control which GPUs are visible to Ray
#os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3,4,5,6"

# Initialize Ray with the appropriate configuration
ray.init(num_gpus=1)


2023-05-18 11:40:07,665	INFO worker.py:1616 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 


Python version:,3.9.16
Ray version:,2.4.0
Dashboard:,http://127.0.0.1:8265


In [4]:
resources = ray.cluster_resources()
print(resources)


{'object_store_memory': 2216358297.0, 'CPU': 20.0, 'node:172.28.132.37': 1.0, 'memory': 4432716596.0, 'GPU': 1.0}


In [5]:
import numpy as np
import gymnasium as gym
from gym_CTgraph import CTgraph_env
from gym_CTgraph.CTgraph_plot import CTgraph_plot
from gym_CTgraph.CTgraph_conf import CTgraph_conf
from gym_CTgraph.CTgraph_images import CTgraph_images
import json
import argparse
#import json
import random
import matplotlib.pyplot as plt
import timeit

from gym.spaces import Box, Discrete, Dict
from ray.tune.registry import register_env
from ray import tune

from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import argparse
#import gym
from gym.spaces import Discrete, Box
import numpy as np
#import os
import random

import ray
from ray import tune
from ray.rllib.algorithms import ppo
from ray.rllib.env.env_context import EnvContext
from ray.rllib.models import ModelCatalog
from ray.rllib.models.tf.tf_modelv2 import TFModelV2
from ray.rllib.models.tf.fcnet import FullyConnectedNetwork
from ray.rllib.models.torch.torch_modelv2 import TorchModelV2
from ray.rllib.models.torch.fcnet import FullyConnectedNetwork as TorchFC
from ray.rllib.utils.framework import try_import_tf, try_import_torch
from ray.rllib.utils.test_utils import check_learning_achieved
from ray.tune.logger import pretty_print

In [6]:
'''import ray
ray.init()
resources = ray.cluster_resources()
print(resources)'''


'import ray\nray.init()\nresources = ray.cluster_resources()\nprint(resources)'

In [7]:
from ray import tune

In [8]:
def env_creator(env_config={}):
    # fetch the parameters from the json file
    configuration = CTgraph_conf("/home/cocp5/graph_test.json")
    conf_data = configuration.getParameters()
    # print configration data
    print(json.dumps(conf_data, indent=3))
    #env = gym.make('CTgraph-v0')
    # instantiate the maze
    #start = timeit.timeit()
    #env = gym.make('CTgraph-v0')
    #end = timeit.timeit()
    #print(end - start)

    imageDataset = CTgraph_images(conf_data)
    env = gym.make('CTgraph-v1', conf_data=conf_data, images=imageDataset)
    env.reset()

    # get a random path from the maze
    #high_reward_path = np.array(np.random.randint(1, conf_data['graph_shape']['branching_factor'], size=conf_data['graph_shape']['depth']))
    # use this random path to set the path to the high reward. Note that the maze would have already a high_reward_path from the initialisation
    #env.set_high_reward_path(high_reward_path)
    #seed_everything(env)  
    
    return env

In [9]:
register_env("CT-Graph_d2_b2_wp05_crv0_MDP", env_creator)

In [ ]:
from ray import tune

tune.run("PPO",
         # algorithm specific configuration
         config={"env": "CT-Graph_d2_b2_wp05_crv0_MDP",  #
                 "framework": "torch",
                 "seed": 959,
                 "num_gpus": 1,
                 "num_workers": 2,
                 "num_envs_per_worker": 1,
                 #"num_gpus_per_worker": 0,
                 "train_batch_size": 512,
                 "use_critic": True,
                 "use_gae": True,
                 "lambda": 0.99,
                 "sgd_minibatch_size": 64,
                 "num_sgd_iter": 8,
                 "shuffle_sequences": False,
                 "entropy_coeff": 0.00015,
                 "clip_param": 0.1,
                 "grad_clip": 5,
                 "lr": 0.00015,
                 "fp16":True,
                 #"num_cpus_per_worker": 1,
                 #"model": {"custom_model": "pa_model", },
                 "evaluation_interval": 1,
                 "evaluation_num_episodes": 5,
                 "evaluation_config": {
                "render_env": True,},
                 },
         local_dir="CT-Graph_d2_b2_wp05_crv0_MDP04",  # directory to save results
         checkpoint_freq=2,  # frequency between checkpoints
         checkpoint_at_end=True,  # save checkpoint at the end of training
         keep_checkpoints_num=6,
         stop={"training_iteration": 500}
         )


/home/cocp5/anaconda3/envs/ctGraph/lib/python3.9/site-packages/ray/tune/experiment/experiment.py:170: UserWarning: The `local_dir` argument of `Experiment is deprecated. Use `storage_path` or set the `TUNE_RESULT_DIR` environment variable instead.
  warnings.warn(


2023-05-18 11:40:13,657	WARNING algorithm_config.py:635 -- Cannot create PPOConfig from given `config_dict`! Property fp16 not supported.
2023-05-18 11:40:13,659	WARNING deprecation.py:50 -- DeprecationWarning: `AlgorithmConfig.evaluation(evaluation_num_episodes=..)` has been deprecated. Use `AlgorithmConfig.evaluation(evaluation_duration=.., evaluation_duration_unit='episodes')` instead. This will raise an error in the future!
(PPO pid=71147) 2023-05-18 11:40:18,157	WARNING algorithm_config.py:635 -- Cannot create PPOConfig from given `config_dict`! Property fp16 not supported.
(PPO pid=71147) 2023-05-18 11:40:18,157	WARNING deprecation.py:50 -- DeprecationWarning: `AlgorithmConfig.evaluation(evaluation_num_episodes=..)` has been deprecated. Use `AlgorithmConfig.evaluation(evaluation_duration=.., evaluation_duration_unit='episodes')` instead. This will raise an error in the future!
(PPO pid=71147) 2023-05-18 11:40:18,733	INFO algorithm.py:527 -- Current log_level is WARN. For more inf

(RolloutWorker pid=71223) ---------------------------------------------------
(RolloutWorker pid=71223)              The CT-graph environments             
(RolloutWorker pid=71223) ---------------------------------------------------
(RolloutWorker pid=71223) Reading configuration parameters from  /home/cocp5/graph_test.json
(RolloutWorker pid=71223) {
(RolloutWorker pid=71223)    "general_seed": 3,
(RolloutWorker pid=71223)    "graph_shape": {
(RolloutWorker pid=71223)       "d": 2,
(RolloutWorker pid=71223)       "b": 2,
(RolloutWorker pid=71223)       "p": 0
(RolloutWorker pid=71223)    },
(RolloutWorker pid=71223)    "reward": {
(RolloutWorker pid=71223)       "high_r": 1.0,
(RolloutWorker pid=71223)       "fail_r": 0.0,
(RolloutWorker pid=71223)       "std_r": 0.0,
(RolloutWorker pid=71223)       "reward_distribution": "needle_in_haystack",
(RolloutWorker pid=71223)       "min_static_reward_episodes": 500,
(RolloutWorker pid=71223)       "max_static_reward_episodes": 500,
(Rollout

2023-05-18 11:40:52,112	WARNING worker.py:1986 -- A worker died or was killed while executing a task by an unexpected system error. To troubleshoot the problem, check the logs for the dead worker. RayTask ID: ffffffffffffffff8ceeda56341beaff441f2f8a01000000 Worker ID: 09bd645556fe3e872f67c17142effba90f526fdf03523988e4c0cf54 Node ID: 08a34756112c4aadb9929e2f18e4f33c84721752df1f8ab7e3b290e2 Worker IP address: 172.28.132.37 Worker port: 37615 Worker PID: 71223 Worker exit type: SYSTEM_ERROR Worker exit detail: Worker unexpectedly exits with a connection error code 2. End of file. There are some potential root causes. (1) The process is killed by SIGKILL by OOM killer due to high memory usage. (2) ray stop --force is called. (3) The worker is crashed unexpectedly due to SIGSEGV or other unexpected errors.


(RolloutWorker pid=71378) --------------------------------------------------- [repeated 4x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/ray-logging.html#log-deduplication for more options.)
(RolloutWorker pid=71378)              The CT-graph environments              [repeated 2x across cluster]
(RolloutWorker pid=71378) Reading configuration parameters from  /home/cocp5/graph_test.json [repeated 2x across cluster]
(RolloutWorker pid=71378) { [repeated 2x across cluster]
(RolloutWorker pid=71378)    "general_seed": 3, [repeated 2x across cluster]
(RolloutWorker pid=71378)    "graph_shape": { [repeated 2x across cluster]
(RolloutWorker pid=71378)       "d": 2, [repeated 2x across cluster]
(RolloutWorker pid=71378)       "b": 2, [repeated 2x across cluster]
(RolloutWorker pid=71378)       "p": 0 [repeated 2x across cluster]
(RolloutWorker pid=71378)    }, [repeated 6x across cl

...

(RolloutWorker pid=87336)              The CT-graph environments             
(RolloutWorker pid=87336) --------------------------------------------------- [repeated 2x across cluster]
(RolloutWorker pid=87336) Reward counter: 0, Reward episodes: 1000
(RolloutWorker pid=87336) Changing high reward path.
(RolloutWorker pid=87419) Reading configuration parameters from  /home/cocp5/graph_test.json
(RolloutWorker pid=87419) {
(RolloutWorker pid=87419)    "general_seed": 3,
(RolloutWorker pid=87419)    "graph_shape": {
(RolloutWorker pid=87419)       "d": 2,
(RolloutWorker pid=87419)       "b": 2,
(RolloutWorker pid=87419)       "p": 0
(RolloutWorker pid=87419)    },
(RolloutWorker pid=87419)    "reward": {
(RolloutWorker pid=87419)       "high_r": 1.0,
(RolloutWorker pid=87419)       "fail_r": 0.0,
(RolloutWorker pid=87419)       "std_r": 0.0,
(RolloutWorker pid=87419)       "reward_distribution": "needle_in_haystack",
(RolloutWorker pid=87419)       "min_static_reward_episodes": 500,
(Rol

(raylet) [2023-05-18 14:04:08,018 E 69762 69762] (raylet) node_manager.cc:3071: 1 Workers (tasks / actors) killed due to memory pressure (OOM), 0 Workers crashed due to other reasons at node (ID: 08a34756112c4aadb9929e2f18e4f33c84721752df1f8ab7e3b290e2, IP: 172.28.132.37) over the last time period. To see more information about the Workers killed on this node, use `ray logs raylet.out -ip 172.28.132.37`
(raylet) 
(raylet) Refer to the documentation on how to address the out of memory issue: https://docs.ray.io/en/latest/ray-core/scheduling/ray-oom-prevention.html. Consider provisioning more memory on this node or reducing task parallelism by requesting more CPUs per task. To adjust the kill threshold, set the environment variable `RAY_memory_usage_threshold` when starting Ray. To disable worker killing, set the environment variable `RAY_memory_monitor_refresh_ms` to zero.


(RolloutWorker pid=87419)              The CT-graph environments             
(RolloutWorker pid=87419) --------------------------------------------------- [repeated 2x across cluster]
(RolloutWorker pid=87419) Reward counter: 0, Reward episodes: 1000
(RolloutWorker pid=87419) Changing high reward path.
(RolloutWorker pid=87499) Reading configuration parameters from  /home/cocp5/graph_test.json
(RolloutWorker pid=87499) {
(RolloutWorker pid=87499)    "general_seed": 3,
(RolloutWorker pid=87499)    "graph_shape": {
(RolloutWorker pid=87499)       "d": 2,
(RolloutWorker pid=87499)       "b": 2,
(RolloutWorker pid=87499)       "p": 0
(RolloutWorker pid=87499)    },
(RolloutWorker pid=87499)    "reward": {
(RolloutWorker pid=87499)       "high_r": 1.0,
(RolloutWorker pid=87499)       "fail_r": 0.0,
(RolloutWorker pid=87499)       "std_r": 0.0,
(RolloutWorker pid=87499)       "reward_distribution": "needle_in_haystack",
(RolloutWorker pid=87499)       "min_static_reward_episodes": 500,
(Rol

(raylet) [2023-05-18 14:05:08,020 E 69762 69762] (raylet) node_manager.cc:3071: 1 Workers (tasks / actors) killed due to memory pressure (OOM), 0 Workers crashed due to other reasons at node (ID: 08a34756112c4aadb9929e2f18e4f33c84721752df1f8ab7e3b290e2, IP: 172.28.132.37) over the last time period. To see more information about the Workers killed on this node, use `ray logs raylet.out -ip 172.28.132.37`
(raylet) 
(raylet) Refer to the documentation on how to address the out of memory issue: https://docs.ray.io/en/latest/ray-core/scheduling/ray-oom-prevention.html. Consider provisioning more memory on this node or reducing task parallelism by requesting more CPUs per task. To adjust the kill threshold, set the environment variable `RAY_memory_usage_threshold` when starting Ray. To disable worker killing, set the environment variable `RAY_memory_monitor_refresh_ms` to zero.


(RolloutWorker pid=87499)              The CT-graph environments             
(RolloutWorker pid=87499) --------------------------------------------------- [repeated 2x across cluster]
(RolloutWorker pid=87499) Reward counter: 0, Reward episodes: 1000
(RolloutWorker pid=87499) Changing high reward path.
(RolloutWorker pid=87579) Reading configuration parameters from  /home/cocp5/graph_test.json
(RolloutWorker pid=87579) {
(RolloutWorker pid=87579)    "general_seed": 3,
(RolloutWorker pid=87579)    "graph_shape": {
(RolloutWorker pid=87579)       "d": 2,
(RolloutWorker pid=87579)       "b": 2,
(RolloutWorker pid=87579)       "p": 0
(RolloutWorker pid=87579)    },
(RolloutWorker pid=87579)    "reward": {
(RolloutWorker pid=87579)       "high_r": 1.0,
(RolloutWorker pid=87579)       "fail_r": 0.0,
(RolloutWorker pid=87579)       "std_r": 0.0,
(RolloutWorker pid=87579)       "reward_distribution": "needle_in_haystack",
(RolloutWorker pid=87579)       "min_static_reward_episodes": 500,
(Rol